In [ ]:
import numpy as np 
import pandas as pd 
from sklearn import model_selection
from sklearn.metrics import accuracy_score
from collections import Counter
import keras
from keras.models import Sequential
from keras.layers import Activation, MaxPooling1D, Dropout, Flatten, Reshape, Dense, Conv1D
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
import numpy as np
import pandas as pd
import seaborn as sns
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
#reading in the dataset as a Pandas dataframe variable
df = pd.read_csv('/kaggle/input/final-opcodes/all_data.csv')
df
#this dataset has a lot of extra columns we do not need(File Name, Total Opcodes, mov, push, call...)

In [ ]:
#the families we are classifying
print(df.Family.unique())

In [ ]:
#here we are deleting the extra unecessary columns
df = df.loc[:, df.columns != 'Total Opcodes']
df = df.loc[:, df.columns != 'File Name']

#this is our labels for training
labels = np.asarray(df[['Family']].copy())

#encoding the labels to numbers
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
labels = le.fit_transform(labels)

#removing more unecessary columns
for i in range(31):
    df = df.drop(df.columns[1], axis=1)
    
    
#this is our training data
opcode_sequence = (df.drop(df.columns[0], axis=1))
opcode_sequence

In [ ]:
#normalizing the data
from sklearn import preprocessing
normalized_X = preprocessing.normalize(opcode_sequence)
opcode_sequence = normalized_X
opcode_sequence


In [ ]:
#the shape of the data right now
print(opcode_sequence.shape)


In [ ]:
#the 1D CNN CONV1D input layer needs to take in a shape of a 3 by 1 array so we reshape it
opcode_sequence = tf.reshape(opcode_sequence, (9725, 1000, 1))
print(opcode_sequence.shape)

In [ ]:
#plot the accuracy and the validation accuracy
def plot_acc(h):
    plt.plot(h.history['accuracy'])
    plt.plot(h.history['val_accuracy'])

    plt.title('model accuracy')
    plt.ylabel('accuracy and loss')
    plt.xlabel('epoch')

    plt.legend(['acc', 'val acc' ], loc='upper left')
    plt.show()

In [ ]:
#plot the loss and validation loss
def plot_loss(h):
    plt.plot(h.history['loss'])
    plt.plot(h.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('accuracy and loss')
    plt.xlabel('epoch')

    plt.legend(['loss', 'val loss' ], loc='upper left')
    plt.show()

We are now done preprocessing our dataset and ready to start training the models

In [ ]:

model = Sequential()
#the shape of the input is (9725, 1000, 1), where there are 9725 training samples and each training sample has 1k features
model.add(Conv1D(filters= 128, kernel_size=3, activation ='relu',strides = 2, padding = 'valid', input_shape= (1000, 1))) #not sure about a good filter/kernel size
model.add(Conv1D(filters= 256, kernel_size=5, activation ='relu',strides = 2, padding = 'valid'))
model.add(Dense(128*4)) #not sure what number to use
model.add(Activation('relu'))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(21)) #for 21 families
model.add(Activation('softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
#now to train the model:
history = model.fit(opcode_sequence, labels, epochs = 30, validation_split = 0.1, batch_size = 64, shuffle = True)


In [ ]:
#plot the acc and loss graphs
plot_acc(history)
plot_loss(history)

Below are a few more models I am experimenting with

In [ ]:
def calc_conv1d_out(self, l_in, padding, dilation, kernel_size, stride):
        return int(((l_in + 2 * padding - dilation * (kernel_size - 1) - 1) / stride) + 1)

In [ ]:
#another model I am testing out


model = Sequential()

model.add(Conv1D(filters= 32, kernel_size=16, activation ='relu',strides = 2, padding = 'valid', input_shape= (1000, 1)))

model.add(Conv1D(filters= 32, kernel_size=8, activation ='relu',strides = 2, padding = 'valid'))

output2 = calc_conv1d_out(1000, 32, 2, 1, 8, 2)

model.add(Dense(output2))
model.add(Dense(128*4))


model.add(Dense(128))
model.add(Flatten())

model.add(Dense(21)) #for 21 families
model.add(Activation('softmax'))

#print(output2)

opt = keras.optimizers.Adam(learning_rate=0.0001)


model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


In [ ]:
history = model.fit(opcode_sequence, labels, epochs = 30, validation_split = 0.1, batch_size = 64, shuffle = True)

In [ ]:
plot_acc(history)
plot_loss(history)